## Notebook for analysis of wrong analog 2 values

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

import os
import sys
module_path = os.path.abspath(os.path.join('./../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.state_comparator.sensor_data_preparator import load_and_prepare_sensor_data, create_epanet_pressure_df
from  src.state_comparator.comparator_functions import compare_real_data_with_simulated
import src.configfile as config

In [ ]:
prepared_data_dict = load_and_prepare_sensor_data("./../../data/sensor_data/", config.SENSOR_FILES, config.PUMP_FILES)
"""
sensor_names_dict["braila_pressure5770.csv"] = "SenzorComunarzi-NatVech"
sensor_names_dict["braila_pressure5771.csv"] = "SenzorComunarzi-castanului"
sensor_names_dict["braila_pressure5772.csv"] = "SenzorChisinau-Titulescu"
sensor_names_dict["braila_pressure5773.csv"] = "SenzorCernauti-Sebesului"
pump_names_dict["braila_flow211206H360.csv"] = "Jonctiune-3974"     # ("Apollo", "Jonctiune-3974"), ,
pump_names_dict["braila_flow211106H360.csv"] = "Jonctiune-J-3"      # ("GA-Braila", "Jonctiune-J-3")
pump_names_dict["braila_flow318505H498.csv"] = "Jonctiune-J-19"     # ("RaduNegru2", "Jonctiune-J-19")
pump_names_dict["braila_flow211306H360.csv"] = "Jonctiune-2749"     # ("RaduNegruMare", "Jonctiune-2749")
"""
# display(prepared_data_dict["Jonctiune-3974"])
display(prepared_data_dict["SenzorComunarzi-castanului"])

## Values at which sensors were zero

In [ ]:
# ("Apollo", "748-B"), ,
# ("GA-Braila", "751-B")
# ("RaduNegruMare", "763-B")
# ("RaduNegru2", "760-B")

sensor_names = ["Jonctiune-3974", "Jonctiune-J-3", "Jonctiune-J-19", "Jonctiune-2749", "SenzorComunarzi-NatVech", 
                "SenzorComunarzi-castanului", "SenzorChisinau-Titulescu", "SenzorCernauti-Sebesului"]
for node_name in sensor_names:
    print("---------------------------------------------------")
    print("---------------------------------------------------")
    print("Node: ", node_name)
    prepared_data_dict[node_name] = prepared_data_dict[node_name].loc[(prepared_data_dict[node_name].index 
                                                                       > pd.to_datetime("2020-12-14 10:00:00") )]
    
    negative_data = prepared_data_dict[node_name][(prepared_data_dict[node_name]["value"] < 0)]  
    nan_data = prepared_data_dict[node_name][(prepared_data_dict[node_name]["value"].isna() == True)]
    display("negative values", negative_data)
    display("nan values", nan_data)

## Comparison between simulated and real

In [ ]:
day_which_to_compare = "2021-04-12"

epanet_simulated_df = create_epanet_pressure_df("./../../data/epanet_networks//RaduNegru24May2021_2.2.inp", 
                                                selected_nodes=sensor_names)

In [ ]:
sensor_day_df = pd.DataFrame()

for sensor_key in prepared_data_dict:
    temp_df = prepared_data_dict[sensor_key]
    specific_day_df = temp_df[temp_df.index.floor('D') == day_which_to_compare]

    if len(specific_day_df.index) != 24:
        raise Exception("Not enough values for specified day ! Name: " + sensor_key)
        
    df_hours = specific_day_df.index.hour
    specific_day_df.index = df_hours

    # Check if arrays contain NaN values
    if np.isnan(np.sum(specific_day_df.to_numpy().flatten())):
        raise Exception("Data for simulated day contains NaN values, please choose another day !")
        
    sensor_day_df[sensor_key] = pd.Series(specific_day_df.to_numpy().flatten(), index=specific_day_df.index)
    
reordered_df = sensor_day_df.filter(epanet_simulated_df.columns)

In [ ]:
print("Day: " + day_which_to_compare)
display("Simulated ", epanet_simulated_df)
display("Real ", reordered_df)

In [ ]:
compare_real_data_with_simulated(prepared_data_dict, epanet_simulated_df, day_which_to_compare="2021-04-12")